In [1]:
import psycopg2
import pandas as pd
import cuemath_db as cuedb
import config_update_script as cnfg
from update_from_cuedb import *
import math
import datetime
import uuid
import pymongo
from numpy import argsort
import pytz
import json
from pymongo import UpdateOne
from update_db_script import *

local_tz = pytz.timezone('Asia/Kolkata')

last_sync_dt = datetime.datetime(2018, 5, 10, 0, 4, 57, 711220)
print("CUEMATH DB Access Started")
df_activities = IntelDB.get_worksheet_db_data()
# print(df_activities[df_activities['id']== '568ed03a-131f-11e9-a40a-4a000536b220'])
df_node_schema = IntelDB.get_node_schema()
df_student_attempts = IntelDB.get_student_attempts_data(last_sync_dt)
df_student_nodes = IntelDB.get_student_node_data(last_sync_dt)
print("CUEMATH DB Access Ended")

CUEMATH DB Access Started
CUEMATH DB Access Ended


In [5]:
def get_student_chapter_data():
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    std_chp_cols = ['id', 'student_id', 'status', 'assessment_status', 'chapter_version_id', 'started_on', 'completed_on']
    chp_ver_cols = ['chapter_id', 'version']
    chp_cols = ['name', 'code', 'description', 'is_default']
    sql_query = "SELECT "+ ", ".join(['service_concepts_student_chapter.'+s for s in std_chp_cols]) + ", " + \
                            ", ".join(['service_concepts_chapter_version.'+s for s in chp_ver_cols]) + ", " + \
                            ", ".join(['service_concepts_chapter.'+s for s in chp_cols]) + \
                            " FROM service_concepts_student_chapter"+ \
                            " inner join service_concepts_chapter_version on (service_concepts_student_chapter.chapter_version_id = service_concepts_chapter_version.id)" + \
                            " inner join service_concepts_chapter on (service_concepts_chapter_version.chapter_id = service_concepts_chapter.id);"
    print(sql_query)
    cur.execute(sql_query)
    df_module_id_code = pd.DataFrame(cur.fetchall(), columns=std_chp_cols + chp_ver_cols + chp_cols)
    cur.close()
    conn.close()
    return(df_module_id_code)


In [134]:
students = student_collection.collection.find()
with open('student_list.csv', "w") as fp:
    fp.write("Student ID, Name \n")
for student in students:
    with open('student_list.csv', "a") as fp:
        fp.write(student['_id']+', '+ student.get('first_name', '') + " " + student.get('last_name', '') +'\n' )

In [131]:
def get_tags_data():
    conn = cuedb.get_cuemath_redshift_connection()
    cur = conn.cursor()
    entity_tag_cols = ["entity_id", "tag_id", "entity_type"]
    entity_tag_cols = ['ent.' + et for et in entity_tag_cols]
    tag_cols = ["id", "meta_data", "type", "code", "name"]
    tag_cols = ['tg.' + tg for tg in tag_cols]
    sql_query = "SELECT "+ ", ".join(entity_tag_cols) \
                        + ", " + ", ".join(tag_cols) \
                        + " FROM fnrenames.service_concepts_cue_entity_tag ent" \
                        + " INNER JOIN fnrenames.service_concepts_cue_tag tg on (ent.tag_id = tg.id);"
    print(sql_query)
    cur.execute(sql_query)
    df_remedials = pd.DataFrame(cur.fetchall(), columns = entity_tag_cols + tag_cols)
    cur.close()
    conn.close()
    return(df_remedials)

def get_node_schema():
        conn = cuedb.get_cuemath_redshift_connection()
        cur = conn.cursor()
        node_cols = ["id", "created_on", "updated_on", "chapter_id",
                    "worksheet_id", "course_type", "is_live",
                    "is_timed", "complete_in", "sheet_time",
                    "points_available", "complete_criteria",
                    "reattempt_criteria", "unlock_criteria",
                    "position", "attempt_location"
                    ]
        node_cols = ["node."+ w for w in node_cols]
        chapter_cols = ["id", "name", "code", "description",
                    "is_default", "position", "program_id", "version", "is_live",
                    "latest_chapter_id"]
        chapter_cols = ['chapter.'+ c for c in chapter_cols]
        program_cols = ["id", "name", "code", "description", "grade", "is_live"]
        program_cols = ['program.'+p for p in program_cols]
        sql_select = "SELECT " + ", ".join(node_cols) \
                    + ", " + ", ".join(chapter_cols) \
                    + ", " + ", ".join(program_cols) \
                    + " FROM fnrenames.service_concepts_cue_node node LEFT JOIN fnrenames.service_concepts_cue_chapter chapter on (node.chapter_id = chapter.id)" \
                    + " LEFT JOIN fnrenames.service_concepts_cue_program program on (chapter.program_id = program.id)";
        cur.execute(sql_select)
        df = pd.DataFrame(cur.fetchall(), columns = node_cols + chapter_cols + program_cols )
        cur.close()
        conn.close()
        return(df)
    
def gen_dict_extract(key, var):
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):
                for result in gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, list):
                for d in v:
                    for result in gen_dict_extract(key, d):
                        yield result
                        
def get_student_chapter_data():
        conn_intel = cuedb.get_cuemath_redshift_connection()
        cur_intel = conn_intel.cursor()
        # Add chapter.expected_time_complete, chapter.,made_live_on, 
        sql_query = "SELECT std_chap.program_id, std_chap.entity_id, std_chap.entity_type, std_chap.user_id, std_chap.created_on, \
                        chapter.code, chapter.name, chapter.position, chapter.version, std_chap.started_on, std_chap.completed_on \
                     FROM fnrenames.service_concepts_user_entity std_chap left join fnrenames.service_concepts_cue_chapter chapter \
                     on (std_chap.entity_id = chapter.id)\
                     WhERE (std_chap.entity_type = 'CHAPTER' and chapter.is_live='true') or std_chap.entity_type = 'PROGRAM';"
        cur_intel.execute(sql_query)
        df_student_program = pd.DataFrame(cur_intel.fetchall(), columns=['program_id', 'chapter_id', 'entity_type', 'user_id', 'created_on',
                                                                         'chapter_code', 'chapter_name', 'chapter_position', 
                                                                         'chapter_version', 'started_on', 'completed_on'])
        cur_intel.close()
        conn_intel.close()
        return(df_student_program)

In [132]:
df = get_student_chapter_data()

In [133]:
df[df['entity_type'] == 'PROGRAM']

,program_id,chapter_id,entity_type,user_id,created_on,chapter_code,chapter_name,chapter_position,chapter_version,started_on,completed_on
1,ae5c6ede-fdec-11e8-ba25-c4b301c785cf,ae5c6ede-fdec-11e8-ba25-c4b301c785cf,PROGRAM,3ea66c02-6b2d-11e9-8b8f-02420a00f779,2019-04-30 15:19:28.227963,None,None,NaN,NaN,NaT,NaT
3,369f7a8a-c545-11e8-ac94-c4b301c785cf,369f7a8a-c545-11e8-ac94-c4b301c785cf,PROGRAM,a7dd3df4-6b14-11e9-a7fc-02420a00f77a,2019-04-30 12:23:27.309615,None,None,NaN,NaN,NaT,NaT
4,369f7a8a-c545-11e8-ac94-c4b301c785cf,369f7a8a-c545-11e8-ac94-c4b301c785cf,PROGRAM,1f4a9a62-4a41-11e9-aef2-02420a00f913,2019-04-30 15:15:03.567593,None,None,NaN,NaN,NaT,NaT
6,369f7a8a-c545-11e8-ac94-c4b301c785cf,369f7a8a-c545-11e8-ac94-c4b301c785cf,PROGRAM,78fefe90-d138-11e8-944d-02420a000033,2018-10-24 15:53:32.309816,None,None,NaN,NaN,2018-10-24 15:53:32.300985,NaT
13,3698ca9e-c545-11e8-97ff-c4b301c785cf,3698ca9e-c545-11e8-97ff-c4b301c785cf,PROGRAM,46fb3052-e336-11e8-a044-02420a0000a5,2019-04-27 18:41:57.427064,None,None,NaN,NaN,NaT,NaT
14,996ee7a8-65df-11e9-9158-0242ac120005,996ee7a8-65df-11e9-9158-0242ac120005,PROGRAM,f4f7dbae-6a43-11e9-942e-02420a00f770,2019-04-29 11:29:31.821997,None,None,NaN,NaN,NaT,NaT
15,996ee7a8-65df-11e9-9158-0242ac120005,996ee7a8-65df-11e9-9158-0242ac120005,PROGRAM,149e543a-6a45-11e9-88f0-02420a00f77b,2019-04-29 11:37:35.342281,None,None,NaN,NaN,NaT,NaT
17,3681e824-c545-11e8-9db1-c4b301c785cf,3681e824-c545-11e8-9db1-c4b301c785cf,PROGRAM,75ccb8c0-fafa-11e8-ab68-02420a0000da,2018-12-08 22:08:43.055256,None,None,NaN,NaN,2018-12-08 22:08:43.051949,NaT
18,752f8b8a-2701-11e9-b7e1-c4b301c785cf,752f8b8a-2701-11e9-b7e1-c4b301c785cf,PROGRAM,acee516e-d6d4-11e8-944d-02420a000033,2019-04-29 14:33:23.336506,None,None,NaN,NaN,NaT,NaT
19,996ee7a8-65df-11e9-9158-0242ac120005,996ee7a8-65df-11e9-9158-0242ac120005,PROGRAM,dba48ff8-6a69-11e9-bffe-02420a00f778,2019-04-29 16:00:50.212233,None,None,NaN,NaN,NaT,NaT


In [97]:
criteria_dict = json.loads(criteria)
next(gen_dict_extract('node_id', criteria_dict))


'd21eccd8-72b8-4dc6-a273-3f7f08420268'

# Reinsert Sessions that had errors

In [34]:

def retrieve_learnosity_reinsert_sessions(inteldb, sessions_received, df_activities, df_student_nodes, df_student_attempts, df_node_schema):
    '''
    # Given all the activities from before, (retrieve from Mongo) get any sessions. Return parsed sessions data
    # Also, update the students collection with newly inserted sessions. (Also filter out demo sessions.)
    '''
    session_objs, session_updates = [], []
    av_cache = {}
    item_updates, activity_updates, classroom_updates, student_updates = [], [], [], []
    item_response_objs, item_response_updates = [], []
    session_incorrect_objs = []
    for idx, session in enumerate(sessions_received):
        #if(idx<start):
        #	continue
        if(len(item_updates) >= write_batch_size):
            result = inteldb.item_collection.collection.bulk_write(item_updates, ordered=True)
            print("UPDATED ITEMS:", result.modified_count)
            item_updates = []
        if(len(activity_updates) >= write_batch_size):
            result = inteldb.activity_collection.collection.bulk_write(activity_updates, ordered=True)
            print("UPDATED ACTIVITIES:", result.modified_count)
            activity_updates = []
        if(len(classroom_updates) >= write_batch_size):
            result = inteldb.classroom_collection.collection.bulk_write(classroom_updates, ordered=True)
            print("UPDATED CLASSROOMS:", result.modified_count)
            classroom_updates = []
        if(len(student_updates) >= write_batch_size):
            result = inteldb.student_collection.collection.bulk_write(student_updates, ordered=True)
            print("UPDATED STUDENTS:", result.modified_count)
            student_updates = []
#         try:
        check = check_user_authenticity(inteldb, df_student_attempts, session['user_id'], session['activity_id'], session['session_id'])
        print("Session Number:", idx, session.get('session_id'), session.get('user_id'), len(activity_updates), check)
        if(check):
        # Above Statement Checks for dummy users and does not consider their responses
            try:
                # TODO: Get Other data: Assignment, Points Earned, Completed, pass criteria, valid_criteria
                print('DEBUG 1')
                student_node_row = df_student_nodes[(df_student_nodes['worksheet_id'] == session['activity_id']) & \
                                                    (df_student_nodes['user_id'] == session['user_id'])]
                student_attempt_row = df_student_attempts[df_student_attempts['id'] == session['session_id']]
                #HOW TO GET NODE_ID
                node_linked = None
                node_id_set = set(list(student_node_row['node_id']))
                if(len(node_id_set) == 1):
                    node_id = node_id_set.pop()
                    node_linked = inteldb.node_collection.collection.find_one({'_id':node_id})
                else:
                    #TODO: Address this particular case
                    nodes_linked_options = inteldb.node_collection.collection.find({'_id':{'$in':list(node_id_set)}})
                    for nd in nodes_linked_options:
                        if(nd.get('activity_id') ==session['activity_id']):
                            node_linked = nd
#                     raise Exception("Student contains various nodes with same worksheet in them")
                print('DEBUG 2')
                if(node_linked is None):
                    print("KeyError: NODE NOT FOUND", session['session_id'], session['dt_started'], session['activity_id'], node_id_set)
                if(node_linked is not None and student_attempt_row.shape[0] != 0):
                    try:
                        print('DEBUG 3')
                        
                        session_parsed, item_responses_parsed, date_session_dict, item_updates, activity_updates = \
                            parse_sessions(inteldb, session, node_linked, student_node_row.to_dict(orient='records')[0], student_attempt_row.to_dict(orient='records')[0], item_updates, activity_updates)
                    except KeyError:
                        print('Activity not Found: ', session.get('activity_id'))
                        continue
                    session_updates.append(UpdateOne({'_id': session_parsed['_id']}, {'$set': session_parsed}, upsert = True))
                    item_response_updates += [UpdateOne({'_id': item_response['_id']}, {'$set': item_response}, upsert = True) for item_response in item_responses_parsed]
                    av_cache, classroom_updates, student_updates = populate_student_side(inteldb, node_linked, session_parsed, av_cache, classroom_updates, student_updates)
                    print('DEBUG 4')
                    
                    if(len(session_updates) > cnfg.bulk_write_batch_size):
                        print("Writing Data so far 1:", len(session_updates), len(item_response_updates))
                        yield(session_updates, item_response_updates, -1)
                        session_updates, item_response_updates = [], []
                        print("Writing Data so far 2:", len(session_updates), len(item_response_updates))
            except IndexError:
                #print(session)
                print("INDEX_ERROR", session['user_id'], session['activity_id'], session['session_id'])
#         except Exception as e:
#             session['parse_error'] = e
#             session_incorrect_objs.append(session)
    if(len(item_updates) > 0):
        result = inteldb.item_collection.collection.bulk_write(item_updates, ordered = True)
        print("UPDATED ITEMS:", result.modified_count)
        item_updates = []
    if(len(activity_updates) > 0):
        result = inteldb.activity_collection.collection.bulk_write(activity_updates, ordered = True)
        print("UPDATED ACTIVITIES:", result.modified_count)
        activity_updates = []
    if(len(classroom_updates) > 0):
        result = inteldb.classroom_collection.collection.bulk_write(classroom_updates, ordered = True)
        print("UPDATED CLASSROOMS:", result.modified_count)
        classroom_updates = []
    if(len(student_updates) > 0):
        result = inteldb.student_collection.collection.bulk_write(student_updates, ordered = True)
        print("UPDATED STUDENTS:", result.modified_count)
        student_updates = []
    yield(session_updates, item_response_updates, 0)




In [35]:
last_sync_dt = datetime.datetime(2018, 5, 10, 0, 4, 57, 711220)
print("CUEMATH DB Access Started")
df_activities = IntelDB.get_worksheet_db_data()
# print(df_activities[df_activities['id']== '568ed03a-131f-11e9-a40a-4a000536b220'])
df_node_schema = IntelDB.get_node_schema()
df_student_attempts = IntelDB.get_student_attempts_data(last_sync_dt)
df_student_nodes = IntelDB.get_student_node_data(last_sync_dt)
print("CUEMATH DB Access Ended")

CUEMATH DB Access Started
CUEMATH DB Access Ended


In [40]:
fl_nm = './ProductionDB/DB_Insert_Issues/Sessions/db_to_reinsert_2019_08_19.json'
sessions_list = []
with open(fl_nm, "r") as fp:
    sessions = json.load(fp)
for session in sessions.get('Failed_Sessions'):
    del session['parse_error']
    sessions_list.append(session)
print([session.get('session_id') for session in sessions_list])

[]


In [26]:
write_batch_size = cnfg.db_write_batch_size
inteldb = IntelDB('prod')
print(type(inteldb))
generator_retrieve_learnosity_session_data = retrieve_learnosity_reinsert_sessions(inteldb, sessions_list, df_activities, df_student_nodes, df_student_attempts, df_node_schema)
new_session_updates, new_item_response_updates, num_failed_sessions = next(generator_retrieve_learnosity_session_data)
insert_gracefully(inteldb.session_collection, new_session_updates, ordered=True)
insert_gracefully(inteldb.item_response_collection, new_item_response_updates, ordered=True)

<class 'update_from_cuedb.IntelDB'>
NOT IN attempt_db DB: user f82c1aae-8429-11e9-ad18-02420a00f013 activity ef487d31-1103-416c-b70c-fda326239fdf session_id fd23159e-146b-4d46-91a4-0729cdd78263
Session Number: 0 fd23159e-146b-4d46-91a4-0729cdd78263 f82c1aae-8429-11e9-ad18-02420a00f013 0 False
NOT IN attempt_db DB: user f82c1aae-8429-11e9-ad18-02420a00f013 activity 8e7d0730-4e1b-4e49-b8fe-307d1d33d747 session_id 0ea285d8-4717-46b0-a4ae-c7763917bab0
Session Number: 1 0ea285d8-4717-46b0-a4ae-c7763917bab0 f82c1aae-8429-11e9-ad18-02420a00f013 0 False


InvalidOperation: No operations to execute

In [34]:
undone_acts = ['806573d4-a87b-11e9-b228-02420a0003f0', 
'ed67f136-8c1e-11e9-8d89-0242ac12000c', 
'ed7d453e-8c1e-11e9-9a6a-0242ac12000c', 
'c2e39ab6-9642-11e9-87f6-02420a0003b2',
'ed845508-8c1e-11e9-811e-0242ac12000c',
'ed763336-8c1e-11e9-a6f5-0242ac12000c',
'ed9330ba-8c1e-11e9-8f52-0242ac12000c']

# Insert Activity 

In [10]:
last_sync_dt = datetime.datetime(2018, 1, 10, 0, 4, 57, 711220)
worksheet_ids = ['2f0b4434-99a3-11e9-9b56-0242ac120002',
                ]
set_inteldb('prod')
directory='./ProductionDB/'
inteldb = IntelDB('prod')
print("CUEMATH DB Access Started")
df_activities = IntelDB.get_worksheet_db_data()
# print(df_activities[df_activities['id']== '568ed03a-131f-11e9-a40a-4a000536b220'])
df_node_schema = IntelDB.get_node_schema()
df_tags = IntelDB.get_tags_data()
print("CUEMATH DB Access Ended")
for worksheet_id in worksheet_ids:
    ws, nd = update_activities(directory, df_activities[df_activities['id']==worksheet_id], 
                  df_node_schema[df_node_schema['node.worksheet_id']==worksheet_id], 
                  df_tags, last_sync_dt)
    insert_gracefully(inteldb.node_collection, nd)
    insert_gracefully(inteldb.activity_collection, ws)

CUEMATH DB Access Started
CUEMATH DB Access Ended
Starting Activity Collection
updated_worksheets:  1 1
updated_nodes:  1 1
Creating Activity Request
Creaating  Multiplication-4-Digit-Numbers-A01
Activity Saved USCC.G3.MULDIV.V1.B1.A1
TOTAL WORKSHEETS:  1 0 2 1 ./ProductionDB/DB_Insert_Issues/Activities/db_to_reinsert_2019_08_08.json
wrote  2 objects 0 0 0 2
wrote  1 objects 0 0 1 0


# Migrate Student Program Data

In [54]:
student_id = 'f04e4618-62b4-11e9-a9dd-02420a00f76c'
inteldb = IntelDB('local')
student = inteldb.student_collection.collection.find_one({'_id':student_id})
for programID, program in student.get('progress').items():
    if(program.get('program_code')=='CBSE.G8'):
        prgID = programID
        dt_jnd = program.get('dt_joined')
        dt_assigned = program.get('started_on')
        chps_attempted_g8 = program.get('chapters_attempted')
    else:
        prgIDG9 = programID
        chps_attempted_g9 = program.get('chapters_attempted')
        prg_code = program.get('program_code')
        prg_name = program.get('program_name')

chps_attempted_g9.update(chps_attempted_g8)
print(dt_jnd, dt_assigned, chps_attempted_g8, chps_attempted_g9)

inteldb.student_collection.collection.update_one({'_id':student_id}, {'$set':{'progress.'+prgIDG9:
                                                                      {
                                                                        'dt_joined':dt_jnd,
                                                                        'started_on':dt_assigned,
                                                                        'chapters_attempted':chps_attempted_g9,
                                                                        'program_code':prg_code,
                                                                        'program_name':prg_name,
                                                                        'completed_on':None
                                                                      }
                                                                     }})


2019-04-30 11:16:37.884928 None {'65894717-2ac0-4178-b67d-0f4a4ff501be': {'dt_assigned': '2019-05-24 14:50:02.979208', 'chapter_code': 'CBSE.G8.SQRRTS', 'chapter_name': 'Squares and Square Roots', 'chapter_position': 4.0, 'chapter_version': 1.0, 'started_on': '2019-05-24 14:50:02.938298', 'completed_on': None}, 'b989dca9-b94a-46a2-a276-0be1ef5143d2': {'dt_assigned': '2019-04-30 16:26:17.447050', 'chapter_code': 'CBSE.G8.RATNUM', 'chapter_name': 'Rational Numbers', 'chapter_position': 2.0, 'chapter_version': 1.0, 'started_on': '2019-04-30 16:26:17.437388', 'completed_on': None}} {'9292e823-101d-4a82-927b-33fbfd87dbb1': {'dt_assigned': '2019-04-23 16:26:28.439717', 'chapter_code': 'CBSE.G7.FRADEC', 'chapter_name': 'Fractions And Decimals', 'chapter_position': 2.0, 'chapter_version': 1.0, 'started_on': '2019-04-23 16:26:28.433279', 'completed_on': None}, '1fac5a2f-b854-4eda-98a2-222885f1c152': {'dt_assigned': '2019-04-20 11:11:14.390266', 'chapter_code': 'CBSE.G7.INTROD', 'chapter_name': 

NameError: name 'student_collection' is not defined

# Make Demo Accounts Live

In [27]:
inteldb = IntelDB('prod')
student_id_list=[
    'f95e5c02-89da-11e9-b0f0-02420a00f13a',
    '620db1d4-93ba-11e9-ab2c-02420a00f0cd',
    'f8b19bfe-9357-11e9-9426-02420a00f13e',
    '63ebcc88-93ba-11e9-9d45-02420a00f0cd',
    'f84d7fcc-9357-11e9-967d-02420a00f018',
    '639160e4-93ba-11e9-bb19-02420a00f0ce',
    '62dc4ef0-93ba-11e9-a6a9-02420a00f0ca',
    '64507bcc-93ba-11e9-9bb8-02420a00f0cb',
    '66a06b80-93ba-11e9-9290-02420a00f0ca',
    '619942a8-93ba-11e9-821b-02420a00f0ce',
    '63393d86-93ba-11e9-b615-02420a00f0cc',
    '663bac7e-93ba-11e9-b126-02420a00f0cb',
    '6510c2b6-93ba-11e9-9e74-02420a00f0cc',
    'f7ec6822-9357-11e9-a40f-02420a00f00e',
    'f940edf6-9357-11e9-83b2-02420a00f13c',
]
students = inteldb.student_collection.collection.find({'_id':{'$in':student_id_list}})
bulk_updates = []
for student in students:
    print(student.get('_id'), students)
    bulk_updates.append(UpdateOne({'_id': student['_id']}, {'$set': {'is_demo':False}}))
insert_gracefully(inteldb.student_collection, bulk_updates, ordered=True)


619942a8-93ba-11e9-821b-02420a00f0ce <pymongo.cursor.Cursor object at 0x1179a94e0>
620db1d4-93ba-11e9-ab2c-02420a00f0cd <pymongo.cursor.Cursor object at 0x1179a94e0>
62dc4ef0-93ba-11e9-a6a9-02420a00f0ca <pymongo.cursor.Cursor object at 0x1179a94e0>
63393d86-93ba-11e9-b615-02420a00f0cc <pymongo.cursor.Cursor object at 0x1179a94e0>
639160e4-93ba-11e9-bb19-02420a00f0ce <pymongo.cursor.Cursor object at 0x1179a94e0>
63ebcc88-93ba-11e9-9d45-02420a00f0cd <pymongo.cursor.Cursor object at 0x1179a94e0>
64507bcc-93ba-11e9-9bb8-02420a00f0cb <pymongo.cursor.Cursor object at 0x1179a94e0>
6510c2b6-93ba-11e9-9e74-02420a00f0cc <pymongo.cursor.Cursor object at 0x1179a94e0>
663bac7e-93ba-11e9-b126-02420a00f0cb <pymongo.cursor.Cursor object at 0x1179a94e0>
66a06b80-93ba-11e9-9290-02420a00f0ca <pymongo.cursor.Cursor object at 0x1179a94e0>
f7ec6822-9357-11e9-a40f-02420a00f00e <pymongo.cursor.Cursor object at 0x1179a94e0>
f84d7fcc-9357-11e9-967d-02420a00f018 <pymongo.cursor.Cursor object at 0x1179a94e0>
f8b1

# Repopulate Activity Data with Sessions

In [33]:
activity_id = 'aa08fcac-70e3-11e9-8d19-0242ac120005'
inteldb = IntelDB('prod')
activity = inteldb.activity_collection.collection.find_one({'_id':activity_id})
sessions = inteldb.session_collection.collection.find({'activity_id':activity_id})
distribution_score = {}
distribution_time = {}
max_score = 0
max_score_list = []
for idx, session in enumerate(sessions):
    print('Session #', idx+1, " user:", session.get('user_id'))
    distribution_score[session.get('user_id')] = session.get('score')
    distribution_time[session.get('user_id')] = session.get('time_total')
    #     if(session.get('max_score') > max_score):
    max_score_list.append(session.get('max_score'))
    #     populate_student_side(av_cache={}, )
print(max_score_list)


Session # 1  user: 783b79ec-7191-11e9-b018-02420a00faad
Session # 2  user: 20c0e0c2-302e-11e9-b06a-02420a00f1a6
Session # 3  user: 80f770ee-36b1-11e9-8f75-02420a00f19e
Session # 4  user: 9bc2475c-302e-11e9-8f9f-02420a00f1a4
Session # 5  user: 40f9b15e-7650-11e9-b2d7-02420a00f043
Session # 6  user: 8601a7de-3e64-11e9-817e-02420a00f1ae
Session # 7  user: 7753a418-72f5-11e9-809a-02420a00fabc
Session # 8  user: 0a5d0472-35d4-11e9-8f75-02420a00f19e
Session # 9  user: 0eac5148-5d0d-11e9-bb34-02420a00f646
Session # 10  user: 9bc2475c-302e-11e9-8f9f-02420a00f1a4
Session # 11  user: 5ee89502-8076-11e9-8f05-02420a00f014
Session # 12  user: 7468d624-7ae7-11e9-baf6-02420a00f0c3
Session # 13  user: 2d7482e8-8425-11e9-8d34-02420a00f012
Session # 14  user: 35bfe5c2-601d-11e9-a2ba-02420a00f654
Session # 15  user: 0c7287ec-845f-11e9-9514-02420a00f015
Session # 16  user: 5d3d0fa6-4d22-11e9-86a2-02420a00fadb
Session # 17  user: cf3b6d88-513b-11e9-b0a8-02420a00fbd7
Session # 18  user: f0b715dc-89f9-11e9-a

In [30]:
max_score = 24
result = inteldb.activity_collection.collection.update_one({ '_id':activity_id }, {'$set':{'distribution_score':distribution_score, 'distribution_time':distribution_time, 'max_score':max_score}})
result.modified_count


1

# Changing DT Joined for Student Progress

In [ ]:
inteldb = IntelDB('prod')
bulk_updates = []
students = inteldb.student_collection.collection.find()
for student in students:
    progress_obj = {}
    prg_code = student.get('current_program')
    if(prg_code == 'CBSE.G7'):
        prg_code = 'G7'
    elif(prg_code == 'CBSE.G8'):
        prg_code = 'G8'
    if(student.get('progress') is None):
        continue
    for prg_id, program_obj in student.get('progress').items():
        old_strted_on = 'NOT HELLO'
        if(program_obj.get('started_on') is None):
            old_strted_on = 'HELLO'
            if(program_obj.get('dt_joined') is None):
                print("BHENCHOD", student.get('first_name') + ' ' + student.get('last_name'))
                continue
            program_obj['started_on'] = program_obj['dt_joined']
#             program_code = 'G7'
#         elif(program_obj.get('program_code') == 'CBSE.G8'):
#             program_code = 'G8'
#         else:
#             program_code = program_obj.get('program_code')
#         program_obj['program_code'] = program_code
        print('DEBUG', program_obj['started_on'], student.get('_id'), old_strted_on)
        progress_obj[prg_id] = program_obj
    bulk_updates.append(UpdateOne({'_id': student['_id']}, {'$set': {'progress':progress_obj, 'current_program':prg_code}}))
    
insert_gracefully(inteldb.student_collection, bulk_updates)     

DEBUG 2019-03-25 10:08:25.654199 d41531f8-4eb7-11e9-99ba-02420a00fadf NOT HELLO
DEBUG 2019-02-14 12:40:22.472162 2c06a296-301f-11e9-97d5-02420a00f1af NOT HELLO
DEBUG 2018-12-05 15:45:47.123111 3100de78-f876-11e8-ba85-02420a0000cc NOT HELLO
DEBUG 2019-04-02 16:35:22.303078 dd9b7e52-2613-11e9-9db9-02420a000860 NOT HELLO
DEBUG 2019-02-18 13:26:59.782543 dd9b7e52-2613-11e9-9db9-02420a000860 NOT HELLO
DEBUG 2019-04-24 15:54:10.333973 22a5e402-5c37-11e9-b58b-02420a00f4b9 NOT HELLO
DEBUG 2019-04-11 14:22:28.688177 22a5e402-5c37-11e9-b58b-02420a00f4b9 NOT HELLO
DEBUG 2019-04-15 10:52:16.537109 6edaf748-5f3e-11e9-aae2-02420a00f654 NOT HELLO
DEBUG 2019-04-08 15:38:00.200591 30426d94-59e6-11e9-91f7-02420a00f461 NOT HELLO
DEBUG 2019-08-16 11:17:39.910441 30426d94-59e6-11e9-91f7-02420a00f461 NOT HELLO
DEBUG 2018-12-17 14:28:50.207474 34729184-01d4-11e9-862f-02420a0000d8 NOT HELLO
DEBUG 2019-04-05 18:13:58.480414 60573e76-579b-11e9-b022-02420a00f48d NOT HELLO
DEBUG 2019-03-19 23:43:06.932151 a4ff952

DEBUG 2019-01-23 22:24:51.012975 174b6c92-1f26-11e9-ae6f-02420a0001d5 NOT HELLO
DEBUG 2019-05-08 17:42:10.457611 174b6c92-1f26-11e9-ae6f-02420a0001d5 NOT HELLO
DEBUG 2019-04-05 18:13:59.663386 72da97ba-579b-11e9-9d55-02420a00f4a2 NOT HELLO
DEBUG 2019-04-15 16:33:03.141776 09d9736c-5f6e-11e9-a7ed-02420a00f654 NOT HELLO
DEBUG 2019-04-03 19:06:51.989512 89c340da-5615-11e9-96dd-02420a00f0cc NOT HELLO
DEBUG 2019-04-08 14:23:35.439876 30323838-36b1-11e9-8214-02420a00f1aa NOT HELLO
DEBUG 2019-02-22 20:20:25.775500 30323838-36b1-11e9-8214-02420a00f1aa NOT HELLO
DEBUG 2018-10-17 15:01:25.024870 d840754c-d1eb-11e8-ad72-02420a000035 NOT HELLO
DEBUG 2019-04-01 15:59:12.286951 d840754c-d1eb-11e8-ad72-02420a000035 NOT HELLO
DEBUG 2018-12-08 22:08:54.880199 96725db8-fafb-11e8-b891-02420a0000d7 NOT HELLO
DEBUG 2019-04-05 18:14:09.965069 41233094-579b-11e9-a17c-02420a00f4a4 NOT HELLO
DEBUG 2019-04-15 11:30:05.342593 b7128002-5f43-11e9-9258-02420a00f665 NOT HELLO
DEBUG 2019-03-20 22:43:40.334204 81af840

DEBUG 2018-12-17 14:27:43.198768 3fe32964-01d1-11e9-862f-02420a0000d8 NOT HELLO
DEBUG 2018-12-10 14:21:39.534914 7304b9a8-fc58-11e8-ab01-02420a0000d5 NOT HELLO
DEBUG 2019-04-08 14:08:09.630246 a3444c38-59d9-11e9-b44f-02420a00f460 NOT HELLO
DEBUG 2019-03-29 11:46:30.074687 3112bfcc-51ea-11e9-961b-02420a00fbe4 NOT HELLO
DEBUG 2019-05-20 12:47:33.040444 9bc2475c-302e-11e9-8f9f-02420a00f1a4 NOT HELLO
DEBUG 2019-02-14 13:30:35.232862 9bc2475c-302e-11e9-8f9f-02420a00f1a4 NOT HELLO
DEBUG 2019-04-05 18:13:57.942806 57c50f5a-579b-11e9-a267-02420a00f460 NOT HELLO
DEBUG 2019-04-15 12:53:22.464840 59b4431c-5f4f-11e9-9113-02420a00f665 NOT HELLO
DEBUG 2019-03-22 16:06:56.616989 69b1bad2-4c8e-11e9-86a2-02420a00fadb NOT HELLO
DEBUG 2019-03-29 17:27:48.697572 69b1bad2-4c8e-11e9-86a2-02420a00fadb NOT HELLO
DEBUG 2019-04-05 18:13:57.897875 56c5846e-579b-11e9-8eee-02420a00f46a NOT HELLO
DEBUG 2019-06-29 00:10:31.574353 56c5846e-579b-11e9-8eee-02420a00f46a NOT HELLO
DEBUG 2019-02-28 14:45:18.427399 937fcc4

# Change Program Codes for chapter

In [40]:
inteldb = IntelDB('prod')
bulk_updates = []
chapters = inteldb.chapter_collection.collection.find({'program_code':'CBSE.G7'})
for chapter in chapters:
    print('chapter', chapter.get('_id'))
    bulk_updates.append(UpdateOne({'_id': chapter['_id']}, {'$set': {'program_code':'G7'}}))
    
insert_gracefully(inteldb.chapter_collection, bulk_updates, ordered=True) 


chapter CBSE.G7.TRIANG
chapter CBSE.G7.AREPEM
chapter CBSE.G7.INTROD
chapter CBSE.G7.POWRTS
chapter CBSE.G7.RATPRO
chapter CBSE.G7.FRADEC
chapter CBSE.G7.COMMTH
chapter CBSE.G7.CONSTR
chapter CBSE.G7.LINANG
chapter CBSE.G7.PRCNTG
chapter CBSE.G7.DATHAN
chapter CBSE.G7.RATNUM
chapter CBSE.G7.SOLSHP
chapter CBSE.G7.INTEGR
chapter CBSE.G7.SYMMTR
chapter CBSE.G7.EXPONT
chapter CBSE.G7.ALGBRA
wrote  17 objects 17 0 0 17


# Add Chapter IDs to Student Session Data

In [14]:
inteldb = IntelDB('prod')
bulk_updates = []
students = inteldb.student_collection.collection.find()
werrors= []
attribs = ['lessons', 'homework', 'assessments']
newVals = {}
for idx, student in enumerate(students):
    newVals['$set'] = {}
    for attrib in attribs:
        out_attrib = {}
        for activity_id, activity_summary in student.get(attrib, {}).items():
            if(activity_summary.get('chapter_id') is not None):
                continue
            student_nds_possible = df_student_nodes[(df_student_nodes['node_id']==activity_summary.get('node_id'))\
                                                    & (df_student_nodes['user_id']==student.get('_id'))]
            student_node_dict = student_nds_possible.to_dict(orient='records')
            if(len(student_node_dict)==1):
                activity_summary['chapter_id'] = student_node_dict[0].get('chapter_id')
            elif(len(student_node_dict)>1):
                print("Such is happening")
            else:
                student_nds_possible = df_student_nodes[(df_student_nodes['worksheet_id']==activity_id)\
                                                    & (df_student_nodes['user_id']==student.get('_id'))]
                student_node_dict = student_nds_possible.to_dict(orient='records')
                if(len(student_node_dict) == 1):
                    activity_summary['chapter_id'] = student_node_dict[0].get('chapter_id')
                elif(len(student_node_dict) == 0):    
                    print("Zero length", attrib, activity_id, activity_summary.get('node_id'), 
                          student.get('first_name') + ' ' + student.get('last_name'))
                    continue
            #             print(activity_id, activity_summary['chapter_id'])
            newVals['$set'].update({attrib+'.'+activity_id:activity_summary})
    if(newVals['$set']!={}):
        print(student.get('first_name') + ' ' + student.get('last_name'), newVals['$set'].keys())
        bulk_updates.append(UpdateOne({'_id':student.get('_id')}, newVals, upsert=True))
    if(len(bulk_updates)>=500):
        try:
            insert_gracefully(inteldb.student_collection, bulk_updates, ordered=True) 
            bulk_updates = []
        except pymongo.errors.BulkWriteError as bwe:
            print(bwe.details)
            werrors.append(bwe.details['writeErrors'])
try:
    insert_gracefully(inteldb.student_collection, bulk_updates) 
except pymongo.errors.BulkWriteError as bwe:
    print(bwe.details)
    werrors.append(bwe.details['writeErrors'])
    

  

Anshika Gupta dict_keys(['lessons', 'homework', 'assessments'])
bb student dict_keys(['lessons', 'homework', 'assessments'])
Aaryaman  dict_keys(['lessons', 'homework', 'assessments'])
demo7 student26 dict_keys(['lessons', 'homework', 'assessments'])
Sudiksha  dict_keys(['lessons', 'homework', 'assessments'])
Nikanshi  dict_keys(['lessons', 'homework', 'assessments'])
Manju  Chandak dict_keys(['lessons', 'homework', 'assessments'])
Adithi Sharan dict_keys(['lessons', 'homework', 'assessments'])
Shweta Dandriyal dict_keys(['lessons', 'homework', 'assessments'])
demo8 student32 dict_keys(['lessons', 'homework', 'assessments'])
demo9 student23 dict_keys(['lessons', 'homework', 'assessments'])
Toshani Konwar dict_keys(['lessons', 'homework', 'assessments'])
demo9 student26 dict_keys(['lessons', 'homework', 'assessments'])
Hallie Vikam dict_keys(['lessons', 'homework', 'assessments'])
Sakshi Milind sodmise dict_keys(['lessons', 'homework', 'assessments'])
Mridula B dict_keys(['lessons', 'ho

wrote  500 objects 0 0 0 500
demo10 student7 dict_keys(['lessons', 'homework', 'assessments'])
Anuradha Chopra dict_keys(['lessons', 'homework', 'assessments'])
Sankar Krishnan dict_keys(['lessons', 'homework', 'assessments'])
demo10 student13 dict_keys(['lessons', 'homework', 'assessments'])
Vedha Pant dict_keys(['lessons', 'homework', 'assessments'])
Wayne Raymond dict_keys(['lessons', 'homework', 'assessments'])
Shanelle  dict_keys(['lessons', 'homework', 'assessments'])
Devang  dict_keys(['lessons', 'homework', 'assessments'])
Sumedh  dict_keys(['lessons', 'homework', 'assessments'])
Grade8b Student dict_keys(['lessons', 'homework', 'assessments'])
demo7 student34 dict_keys(['lessons', 'homework', 'assessments'])
demo9 student3 dict_keys(['lessons', 'homework', 'assessments'])
demo9 student88 dict_keys(['lessons', 'homework', 'assessments'])
Upasana  Mahant dict_keys(['lessons', 'homework', 'assessments'])
demo9 student90 dict_keys(['lessons', 'homework', 'assessments'])
demo9 stud

demo7 student48 dict_keys(['lessons', 'homework', 'assessments'])
demo8 student60 dict_keys(['lessons', 'homework', 'assessments'])
demo8 student50 dict_keys(['lessons', 'homework', 'assessments'])
demo7 student57 dict_keys(['lessons', 'homework', 'assessments'])
Amogh Gupta dict_keys(['lessons', 'homework', 'assessments'])
demo7 student46 dict_keys(['lessons', 'homework', 'assessments'])
Suhita Desai dict_keys(['lessons', 'homework', 'assessments'])
demo8 student73 dict_keys(['lessons', 'homework', 'assessments'])
demo7 student45 dict_keys(['lessons', 'homework', 'assessments'])
demo8 student52 dict_keys(['lessons', 'homework', 'assessments'])
demo7 student55 dict_keys(['lessons', 'homework', 'assessments'])
demo8 student57 dict_keys(['lessons', 'homework', 'assessments'])
demo8 student55 dict_keys(['lessons', 'homework', 'assessments'])
demo7 student77 dict_keys(['lessons', 'homework', 'assessments'])
demo7 student50 dict_keys(['lessons', 'homework', 'assessments'])
demo8 student42 d

wrote  382 objects 0 0 0 382


In [10]:
werrors

[[{'index': 0,
   'code': 9,
   'errmsg': "'$set' is empty. You must specify a field like so: {$set: {<field>: ...}}",
   'op': SON([('q', {'_id': '9dbc91ac-88e9-11e9-b063-02420a00f012'}), ('u', {'$set': {}}), ('multi', False), ('upsert', True)])}]]

In [ ]:
undone_student_ids = ['0f4d1ea4-555f-11e9-a169-02420a00fbdc', 
                      '262f4528-5515-11e9-8da8-02420a00fbd6', 
                      '674abc36-5448-11e9-b70c-02420a00fbdd', 
                      'a51a0e06-d5f8-11e8-a0a7-02420a00002f',
                      '149ba486-515e-11e9-9389-02420a00fbdf',
                      '7c062330-56a7-11e9-b7bb-02420a00f4a4',
                      '0daf4ccc-399c-11e9-9acc-02420a00f1ad',
                      '7c062330-56a7-11e9-b7bb-02420a00f4a4',
                      '6488b9dc-d5f8-11e8-a061-02420a00002c',
                      '20c0e0c2-302e-11e9-b06a-02420a00f1a6',
                      '149ba486-515e-11e9-9389-02420a00fbdf',
                     ]